# 코로나 이전 vs 코로나 기간 동안 vs 코로나 이후의 드라마 시청률 비교
t test활용

In [27]:
import numpy as np
import pandas as pd
import datetime
from scipy import stats


## data preparation

### google colab 사용시

In [28]:
# from google.colab import files
# import io

In [29]:
#미니시리즈_메타데이터.csv파일 업로드

# uploaded = files.upload()
# mini_meta_df = pd.read_csv(io.BytesIO(uploaded['미니시리즈_메타데이터.csv']))



In [30]:
#미니시리즈 전처리1.csv파일 업로드

# uploaded = files.upload()
# mini_preprocessed_df = pd.read_csv(io.BytesIO(uploaded['미니시리즈 전처리1.csv']))


In [31]:
#드라마_시청률_데이터.csv파일 업로드

# uploaded = files.upload()
# rating_df = pd.read_csv(io.BytesIO(uploaded['드라마_시청률_데이터.csv']))


### 로컬에서 불러올시

In [32]:
file_path = r"C:\Users\sjung\Desktop\Data_Science_Lab\EDA_project\10th-EDA\Team_C\정제데이터\\"
rating_d= pd.read_csv(file_path+'전체시청률.csv')
rating_d

,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
0,(아는 건 별로 없지만) 가족입니다,2020-06-01,3.122,2020,6,1,1,1
1,(아는 건 별로 없지만) 가족입니다,2020-06-02,3.930,2020,6,2,2,1
2,(아는 건 별로 없지만) 가족입니다,2020-06-08,3.345,2020,6,8,3,1
3,(아는 건 별로 없지만) 가족입니다,2020-06-09,3.862,2020,6,9,4,1
4,(아는 건 별로 없지만) 가족입니다,2020-06-15,3.188,2020,6,15,5,1
...,...,...,...,...,...,...,...,...
8709,힙합왕 - 나스나길,2019-08-16,1.200,2019,8,16,2,2
8710,힙합왕 - 나스나길,2019-08-23,1.300,2019,8,23,3,2
8711,힙합왕 - 나스나길,2019-08-30,1.350,2019,8,30,4,2
8712,힙합왕 - 나스나길,2019-09-06,0.600,2019,9,6,5,2


## data preprocessing

In [33]:
#드라마별 회차 추출
drama_episode_count = rating_d.groupby('작품명')['시청률'].count().reset_index()
drama_episode_count.rename(columns={'시청률':'count'}, inplace = True)
drama_episode_count.head()

,작품명,count
0,(아는 건 별로 없지만) 가족입니다,16
1,17세의 조건,2
2,18 어게인,16
3,365: 운명을 거스르는 1년,12
4,"60일, 지정생존자",16


In [34]:
#12부작 이상인 드라마만 추출
min_episode = 12
max_episode = np.inf
bool_mask = (min_episode<=drama_episode_count['count']) & (drama_episode_count['count']<=max_episode)
extracted_dama_df = drama_episode_count[bool_mask].reset_index(drop=True)
extracted_dama_df

,작품명,count
0,(아는 건 별로 없지만) 가족입니다,16
1,18 어게인,16
2,365: 운명을 거스르는 1년,12
3,"60일, 지정생존자",16
4,99억의 여자,16
...,...,...
304,홍천기,16
305,화양연화,16
306,환혼,20
307,황금 가면,100


In [35]:
extracted_dama_ratig = rating_d[rating_d['작품명'].isin(extracted_dama_df['작품명'])].reset_index(drop=True)
extracted_dama_ratig


,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
0,(아는 건 별로 없지만) 가족입니다,2020-06-01,3.122,2020,6,1,1,1
1,(아는 건 별로 없지만) 가족입니다,2020-06-02,3.930,2020,6,2,2,1
2,(아는 건 별로 없지만) 가족입니다,2020-06-08,3.345,2020,6,8,3,1
3,(아는 건 별로 없지만) 가족입니다,2020-06-09,3.862,2020,6,9,4,1
4,(아는 건 별로 없지만) 가족입니다,2020-06-15,3.188,2020,6,15,5,1
...,...,...,...,...,...,...,...,...
8361,황금정원,2019-09-28,8.250,2019,9,28,11,4
8362,황금정원,2019-10-05,8.850,2019,10,5,12,4
8363,황금정원,2019-10-12,9.075,2019,10,12,13,4
8364,황금정원,2019-10-19,8.600,2019,10,19,14,4


In [36]:
# '방송일자'열의 dtype이 string으로 나오기 때문에, datetime으로 변환시키기
extracted_dama_ratig['방송일자'] = pd.to_datetime(extracted_dama_ratig['방송일자'])
print(extracted_dama_ratig.dtypes)

작품명             object
방송일자    datetime64[ns]
시청률            float64
방영연도             int64
월                int64
일                int64
회차               int64
연속방송             int64
dtype: object


In [57]:
# 결측치 제거
extracted_dama_ratig = extracted_dama_ratig.dropna(axis = 0).reset_index(drop=True)

In [58]:
extracted_dama_ratig

,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
0,(아는 건 별로 없지만) 가족입니다,2020-06-01,3.122,2020,6,1,1,1
1,(아는 건 별로 없지만) 가족입니다,2020-06-02,3.930,2020,6,2,2,1
2,(아는 건 별로 없지만) 가족입니다,2020-06-08,3.345,2020,6,8,3,1
3,(아는 건 별로 없지만) 가족입니다,2020-06-09,3.862,2020,6,9,4,1
4,(아는 건 별로 없지만) 가족입니다,2020-06-15,3.188,2020,6,15,5,1
...,...,...,...,...,...,...,...,...
8234,황금정원,2019-09-28,8.250,2019,9,28,11,4
8235,황금정원,2019-10-05,8.850,2019,10,5,12,4
8236,황금정원,2019-10-12,9.075,2019,10,12,13,4
8237,황금정원,2019-10-19,8.600,2019,10,19,14,4


## test setting

코로나 시작과 끝 시점을 대한민국에서 사회적 거리두기 시행 기준으로 설정함
- 코로나 시작: 2020.03.22 (강화된 사회적 거리두기)
- 코로나 끝: 2022.04.18 (모든 사회적 거리두기 단계가 마스크 의무화를 제외하고 전면 해제)

In [60]:
covid_begin = datetime.datetime(2020,3,22,0,0,0)
covid_end = datetime.datetime(2022,4,18,0,0,0)

In [61]:
prior_covid_rating = extracted_dama_ratig[extracted_dama_ratig['방송일자'] < covid_begin]
prior_covid_rating

,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
44,"60일, 지정생존자",2019-07-01,3.383,2019,7,1,1,1
45,"60일, 지정생존자",2019-07-02,4.157,2019,7,2,2,1
46,"60일, 지정생존자",2019-07-08,4.336,2019,7,8,3,1
47,"60일, 지정생존자",2019-07-09,4.155,2019,7,9,4,1
48,"60일, 지정생존자",2019-07-15,4.277,2019,7,15,5,1
...,...,...,...,...,...,...,...,...
8234,황금정원,2019-09-28,8.250,2019,9,28,11,4
8235,황금정원,2019-10-05,8.850,2019,10,5,12,4
8236,황금정원,2019-10-12,9.075,2019,10,12,13,4
8237,황금정원,2019-10-19,8.600,2019,10,19,14,4


In [62]:
during_covid_rating = extracted_dama_ratig[(covid_begin <= extracted_dama_ratig['방송일자']) &\
                                            (extracted_dama_ratig['방송일자'] < covid_end)]
during_covid_rating

,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
0,(아는 건 별로 없지만) 가족입니다,2020-06-01,3.122,2020,6,1,1,1
1,(아는 건 별로 없지만) 가족입니다,2020-06-02,3.930,2020,6,2,2,1
2,(아는 건 별로 없지만) 가족입니다,2020-06-08,3.345,2020,6,8,3,1
3,(아는 건 별로 없지만) 가족입니다,2020-06-09,3.862,2020,6,9,4,1
4,(아는 건 별로 없지만) 가족입니다,2020-06-15,3.188,2020,6,15,5,1
...,...,...,...,...,...,...,...,...
8099,화양연화,2020-05-31,3.878,2020,5,31,12,1
8100,화양연화,2020-06-06,4.128,2020,6,6,13,1
8101,화양연화,2020-06-07,3.655,2020,6,7,14,1
8102,화양연화,2020-06-13,4.842,2020,6,13,15,1


In [63]:
post_covid_rating = extracted_dama_ratig[extracted_dama_ratig['방송일자'] >= covid_end]
post_covid_rating

,작품명,방송일자,시청률,방영연도,월,일,회차,연속방송
92,가면의 여왕,2023-04-24,1.401,2023,4,24,1,1
93,가면의 여왕,2023-04-25,1.501,2023,4,25,2,1
94,가면의 여왕,2023-05-01,2.270,2023,5,1,3,1
95,가면의 여왕,2023-05-02,2.413,2023,5,2,4,1
96,가면의 여왕,2023-05-08,2.482,2023,5,8,5,1
...,...,...,...,...,...,...,...,...
8219,황금 가면,2022-10-03,17.100,2022,10,3,96,1
8220,황금 가면,2022-10-04,16.300,2022,10,4,97,1
8221,황금 가면,2022-10-05,15.500,2022,10,5,98,1
8222,황금 가면,2022-10-06,16.700,2022,10,6,99,1


# Hypothesis testing

## 전체 데이터 (장르 구별X)

1. hypothesis test1<br>
```null hypothesis: 코로나 이전이 코로나 기간 동안 보다 시청률이 높거나 같을 것이다.```<br>
(prior_covid_rating >= during_covid_rating)<br>
```alternative hypothesis: 코로나 기간 동안이 코로나 이전보다 시청률이 높을 것이다.```<br>
(prior_covid_rating < during_covid_rating)


In [66]:
# prior_covid_rating vs during_covid_rating
# null hypothesis: prior_covid_rating >= during_covid_rating
# alternative hypothesis: prior_covid_rating < during_covid_rating

t_stat, p_val = stats.ttest_ind(prior_covid_rating['시청률'],
                                during_covid_rating['시청률'],
                                equal_var = False,
                                alternative = 'less'
                                )
print(f"t-statistics: {t_stat}")
print(f"p-value: {p_val}")

t-statistics: -8.067050263913591
p-value: 4.318080758886689e-16


2. hypothesis test2<br>
```null hypothesis: 코로나 이후가 코로나 기간 동안 보다 시청률이 높거나 같을 것이다.```<br>
(during_covid_rating <= post_covid_rating)<br>
```alternative hypothesis: 코로나 기간 동안이 코로나 이후보다 시청률이 높을 것이다.```<br>
(during_covid_rating > post_covid_rating)<br>
왜 그런진 모르겠지만 코로나 동안 vs 코로나 이후 비교는 nan이 나온다...

In [69]:
# during_covid_rating vs post_covid_rating
# null hypothesis: during_covid_rating <= post_covid_rating
# alternative hypothesis: during_covid_rating > post_covid_rating

t_stat, p_val = stats.ttest_ind(during_covid_rating['시청률'],
                                post_covid_rating['시청률'],
                                equal_var = False,
                                alternative = 'greater'
                                )
print(f"t-statistics: {t_stat}")
print(f"p-value: {p_val}")

t-statistics: 8.059367977794338
p-value: 4.903945405325962e-16


## 장르별로 비교
이건 미완성